In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")

In [ ]:
text = """
Fintech founder charged with fraud after ‘AI’ shopping app found to be powered by humans in the Philippines Albert Saniger, the founder and former CEO of Nate, an AI shopping app that promised a “universal” checkout experience, was charged with defrauding investors on Wednesday, according to a press release from the U.S. Department of Justice.
"""

labels = ["person", "location", "date", "event", "legal action", "organization"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


In [ ]:
import os
from openai import AzureOpenAI

endpoint = "endpoint"
model_name = "gpt-4o"
deployment = "gpt-4o-2"

subscription_key = "api key"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """
            You are an expert in extracting named entities from financial news. I will be providing a set of entities and some news articles containing [title] and [snippet], and you will need to extract the entities from the articles. Multiple news articles will be seperated by a newline character.
            ## Constraints
            1. References to the same entity should be grouped together while all variations should be listed.
            2. For each article, the output should be in a dictionary. The dictionary should contain the entities identified from that article, and the fields should include each entity type, entity_name and variations.
            3. Return the exact same number of dictionaries as the number of articles nested in a list. The order of the results should follow the order of the articles.
            4. The output should be a JSON object. Only return the JSON object, nothing else. Do not include any other strings whatsoever, even it's for formatting.

            ## Sample Output
            {
                "COMPANY": [
                    {
                    "entity_name": "...",
                    "variations": [
                        "...",
                    ]
                    },
                    ...
                ],
                "PERSON": [
                    {
                    "entity_name": "...",
                    "variations": [
                        "...",
                    ]
                    },
                    ...
                ],
                "FINANCIAL_INSTITUTION": [...],
                "REGULATORY_BODY": [...],
                "PROTENTIAL_CRIME": [...],
                "LEGAL_ACTION": [...],
                "ENFORCEMENT_ACTION": [...],
                "LOCATION": [...],
                "SANCTION_ENTITY": [...],
                "SECTOR": [...],
                "REGULATION": [...]
                }
            
            ## Entities
            COMPANY: Public or private companies, including subsidiaries.
            PERSON: Individuals, especially executives, board members, or other key figures.
            FINANCIAL_INSTITUTION: Banks, investment firms, hedge funds, etc.
            REGULATORY_BODY: Agencies like SEC, MAS, FCA, etc.
            PROTENTIAL_CRIME: Terms indicating potential crime.
            LEGAL_ACTION: Mentions of lawsuits, indictments, probes, settlements.
            ENFORCEMENT_ACTION: Actions like fines, bans, license suspensions.
            LOCATION: Cities, countries—important for sanctions or jurisdiction context.
            SANCTION_ENTITY: Country or individual sanctioned or under restrictions.
            SECTOR: Industry or economic sector (e.g., "energy", "fintech").
            REGULATION: Names or identifiers of laws or policies (e.g., GDPR, FCPA, Dodd-Frank).
            """
        },
        {
            "role": "user",
            "content": """
            [title]Fintech founder charged with fraud after ‘AI’ shopping app found to be powered by humans in the Philippines [snippet]Albert Saniger, the founder and former CEO of Nate, an AI shopping app that promised a “universal” checkout experience, was charged with defrauding investors on Wednesday, according to a press release from the U.S. Department of Justice.
            """,
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)